# Natural Language Processing : Préprocessing de Texte et TF-IDF

Ce cours de 3 heures représente une **transition fondamentale** de l'analyse de données numériques vers le traitement du langage naturel. Il illustre parfaitement la progression pédagogique : clustering de données structurées → réduction dimensionnelle → **traitement de données textuelles non-structurées**.

Lien vers l'audio : https://audio-records-dsfs.s3.eu-west-3.amazonaws.com/UnSupervisedML/M07D04_DataScience.mp3

## **Introduction et Contextualisation Pédagogique**

### **Révision Synthétique des Concepts Précédents**

Le cours débute par une **récapitulation active** des algorithmes étudiés :[1]

**K-means** : "Algorithme qui fait des **moyennes**" lors du recalcul des centroïdes. Processus itératif avec convergence vers WCSS minimale, mais attention au caractère **mécaniquement décroissant** de WCSS avec l'augmentation de K.

**DBSCAN** : Clustering **basé sur la densité** avec paramètres critiques ε (epsilon) et min_samples. Capacité unique à **détecter des formes arbitraires** (croissants de lune) là où K-means échoue avec ses partitions sphériques.

**PCA** : **Diagonalisation** de la matrice de covariance pour maximiser la variance expliquée. Transformation linéaire préservant l'information essentielle tout en réduisant la dimensionnalité.

Cette révision établit une **continuité conceptuelle** : des données numériques bien structurées vers les données textuelles intrinsèquement non-structurées.

### **Transition vers le NLP**

**Problématique fondamentale** : Comment une machine peut-elle "comprendre" le texte alors qu'elle n'a aucune notion intrinsèque du sens des mots ?[1]

**Réponse du cours** : Un mot n'est pas défini par ses lettres mais par **le contexte dans lequel il est habituellement observé**. Cette philosophie sous-tend toute la démarche NLP moderne.

## **Fondements Théoriques du Natural Language Processing**

### **Définition et Portée**

Le NLP (Natural Language Processing) est défini comme **"la discipline qui construit des machines capables de manipuler le langage humain"**. Mais le cours va plus loin en précisant que le NLP traditionnel se concentre sur les **méthodes de preprocessing** plutôt que sur la compréhension sémantique profonde.[2][1]

### **Distinction NLP vs NLU**

**NLP** : Méthodes de transformation texte → vecteurs exploitables.[1]

**NLU (Natural Language Understanding)** : Conversion audio → texte → vecteurs, incluant la reconnaissance vocale et l'identification des locuteurs.[1]

**Exemple concret du professeur** : Analyser un transcript de cours pour identifier qui pose des questions (étudiant vs professeur) nécessite du NLU avec reconnaissance vocale, pas seulement du NLP textuel.[1]

### **Défis Intrinsèques du Langage**

**Polysémie contextuelle** : "Apple" signifie entreprise technologique ou fruit selon le contexte. Cette ambiguïté nécessite une analyse contextuelle sophistiquée.[1]

**Variations culturelles** : Les concepts politiques s'expriment différemment selon les cultures (États-Unis vs Corée). Les embeddings doivent être **entraînés sur des données spécifiques à chaque langue et culture**.[1]

**Subjectivité sémantique** : Le mot "amour" a des connotations différentes pour une personne de 20 ans vs 60 ans, illustrant que les humains eux-mêmes ne s'accordent pas sur le sens des mots.[1]

## **Architecture de Traitement : Du Texte aux Vecteurs**

### **Problématique de la Vectorisation**

**Contrainte fondamentale** : Les modèles ML ne traitent que des vecteurs numériques. Le défi consiste à transformer des chaînes de caractères en représentations numériques préservant l'information sémantique.[1]

### **Count Vectorizer : Première Approche**

**Principe** : Matrice avec documents en lignes, mots uniques en colonnes, occurrences en valeurs.[1]

**Exemple pratique du cours** :
```
Corpus:
- "Guillaume va à la plage"  
- "Guillaume va à la salle"
- "Guillaume est prof"
```

**Matrice résultante** :
```
           Guillaume  va  à  la  plage  salle  est  prof
Document1      1      1   1   1    1     0    0    0
Document2      1      1   1   1    0     1    0    0
Document3      1      0   0   0    0     0    1    1
```

### **Problème de Sparsité Critique**

**Dimensionalité explosive** : Avec 10 000 documents et 10 000 mots uniques → 100 millions de cellules avec **~99,5% de zéros**.[1]

**Calcul concret du cours** : 50 mots moyens par document sur 10 000 colonnes → **9 950 zéros par ligne** en moyenne.

**Solution technique** : **Sparse Matrix** stockant uniquement les valeurs non-nulles sous forme de dictionnaire.[1]

**Piège fréquent** : Conversion sparse matrix → DataFrame provoque des **erreurs mémoire** sur de gros datasets (150 millions de cellules = ~1.2 GB RAM minimum).

## **Stratégies de Preprocessing Fondamentales**

### **Règles de Filtrage Intelligentes**

**Première règle** : Les mots **communs à tous les documents** ne permettent pas de différencier un document d'un autre → suppression.[1]

**Deuxième règle** : Les mots **présents dans un seul document** n'apportent rien pour le clustering ou la classification → suppression également.[1]

**Logique sous-jacente** : Conserver uniquement les mots ayant un **pouvoir discriminant** optimal.

### **Stop Words : Filtrage Sophistiqué**

**Définition contextuelle** : Mots très fréquents ne portant pas de sens discriminant dans le contexte spécifique.[1]

**spaCy français** : **326 stop words** par défaut. Liste personnalisable selon le domaine :[3]

```python
import spacy
nlp = spacy.load("fr_core_news_lg") 
stop_words = nlp.Defaults.stop_words
stop_words.add("guillaume")  # Ajout contextuel
```

**Stratégie recommandée** : Analyser les mots les plus fréquents **après premier preprocessing**, identifier manuellement ceux à ajouter.[1]

### **Lemmatisation vs Stemming**

**Stemming** : Approche **"brutale"** utilisant des règles heuristiques.[4][1]
- Rapide, faible coût computationnel
- Peut produire des mots inexistants ("caring" → "car")
- Adapté aux moteurs de recherche

**Lemmatisation** : Approche **"intelligente"** avec analyse morphologique.[5][1]
- Lente, coût computationnel élevé  
- Produit des mots valides du dictionnaire
- Adaptée à l'analyse sémantique fine

**Recommandation du cours** : Lemmatisation **uniquement si le budget computationnel le permet**, sinon stemming suffisant pour beaucoup d'applications.[1]

## **TF-IDF : Revolution du Scoring**

### **Limitation Identifiée du Count Vectorizer**

**Problème** : Le simple comptage ignore l'**importance relative** d'un mot dans le corpus global.[1]

**Exemple concret** : "Guillaume" apparaît dans tous les documents → **haute fréquence locale** mais **faible pouvoir discriminant**.

### **Formulation Mathématique Détaillée**

**Term Frequency (TF)** :[6][1]
$$ TF = \frac{\text{occurrences du terme dans le document}}{\text{mots totaux du document}} $$

**Inverse Document Frequency (IDF)** :[6][1]
$$ IDF = \log\left(\frac{\text{documents totaux}}{\text{documents contenant le terme}}\right) $$

**Score final TF-IDF** :[6][1]
$$ \text{TF-IDF} = TF \times IDF $$

### **Exemple Calculé du Cours**

**Configuration** :[1]
- 2 documents total
- Mot "four" : 2 occurrences dans doc 1, absent du doc 2  
- Document 1 : 8 mots total

**Calculs étape par étape** :
1. **TF** = 2/8 = **0,25**
2. **IDF** = log(2/1) = log(2) ≈ **0,69**
3. **TF-IDF** = 0,25 × 0,69 ≈ **0,17**

### **Propriété d'Auto-Pénalisation**

**Cas critique** : Mot présent dans tous les documents.[1]
- IDF = log(n/n) = log(1) = **0**
- TF-IDF = TF × 0 = **0**

**Objectif atteint** : Pénalisation automatique des mots trop fréquents, valorisation des mots localement importants mais globalement rares.

## **Techniques Avancées et Embeddings**

### **spaCy : Analyse Grammaticale Contextuelle**

**Fonctionnalités avancées** :[1]
- Détection de **noms propres** vs noms communs
- Identification des **compléments d'objet direct/indirect**
- **Relations de dépendance** entre mots dans la phrase

**Solution au problème de casse** : Identifier les noms propres **avant** conversion en minuscules, les marquer spécialement (ex: "apple#PROPN").

### **Word Embeddings et Similarité**

**Principe révolutionnaire** : Représentation vectorielle dense basée sur le **contexte d'usage** plutôt que sur la forme des mots.[1]

**Similarité Cosinus** : Mesure l'angle entre vecteurs de mots :[1]
- **Angle faible** → mots sémantiquement proches  
- **90 degrés** → mots orthogonaux (pas de relation)
- **Angle important** → mots opposés

**Application pratique** : Grouper automatiquement "chat", "chats", "félin" par proximité vectorielle.

### **Limites des Embeddings**

**Dépendance aux données d'entraînement** : Embeddings américains ne fonctionnent pas sur du coréen à cause des structures linguistiques différentes.[1]

**Exemple concret** : "Trump" et "chat" pourraient avoir une faible distance cosinus si le corpus d'entraînement contenait beaucoup de comparaisons "Trump se comporte comme un chat".

## **N-grammes : Préservation du Contexte**

### **Nécessité Contextuelle Critique**

**Exemple paradigmatique du cours** :[1]
- "This guy is **dead**" → sentiment **négatif** (unigramme)
- "This guy is **to die for**" → sentiment **positif** (trigramme)

**Problème de la lemmatisation** : "die" et "dead" convergent vers la même racine, perdant la distinction sémantique cruciale.

**Solution** : Capturer les n-grammes pour préserver le contexte sémantique.

### **Explosion Combinatoire**

**Défi technique** : Inclure tous les n-grammes multiplie exponentiellement le nombre de colonnes.[1]

**Stratégie de mitigation** recommandée :[1]
1. Analyser séparément par **classe cible** (positif/négatif/neutre)
2. Identifier les n-grammes **les plus discriminants**
3. N'inclure que ceux ayant un **fort impact différentiel**

**Exemple pratique** : Plutôt que d'inclure tous les trigrammes, analyser lesquels sont surreprésentés dans les sentiments positifs vs négatifs.

## **Considérations Pratiques et Limitations**

### **Trade-offs Computationnels**

**Correction orthographique** : Très efficace mais **prohibitivement lente**. Recommandation : réserver à l'exploration initiale sur échantillons réduits.[1]

**spaCy vs méthodes simples** : Objets NLP riches en attributs mais **lourds en mémoire**. Tester sur sous-échantillons avant application complète.[1]

### **Gestion Mémoire Critique**

**Règle d'or** : Ne **jamais** convertir directement sparse matrix → DataFrame sans vérifier la taille.[1]

**Exemple critique** : 10 000 docs × 15 000 mots = 150 millions de cellules → **crash mémoire** garanti sur machine standard.

### **Approche Itérative Recommandée**

**Workflow standard** :[1]
1. **Preprocessing initial** + analyse vocabulaire
2. **Ajustement** basé sur les statistiques observées  
3. **Re-preprocessing optimisé**
4. **Validation** sur modèle ML

**Philosophie** : "Il n'y a pas de solution miracle", tout dépend du contexte et des contraintes.

## **Perspectives et Évolution vers les LLM**

### **Positionnement des Techniques Traditionnelles**

**Vision nuancée du cours** : Les LLM ne rendent **pas obsolètes** les techniques de preprocessing traditionnelles. Elles restent cruciales pour :[1]

1. **Comprendre les fondements** des modèles modernes
2. **Optimiser les coûts** (preprocessing + petit modèle vs gros LLM)
3. **Contrôler la qualité** des données d'entrée

### **Usage LLM Recommandé**

**Approche pédagogique du professeur** : Utiliser les LLM pour **brainstorming** et **validation méthodologique** plutôt que remplacement complet.[1]

**Exemple concret** : Avant de préparer le cours, utiliser un LLM pour explorer différentes manières d'expliquer les concepts, enrichir les analogies, sourcer les références.

**Principe directeur** : Approche **proactive** plutôt que passive (ne pas juste copier-coller le code généré).

## **Impact Pédagogique et Méthodologique**

Ce cours illustre parfaitement une progression pédagogique en data science : partir de problèmes numériques bien définis (clustering) vers des défis plus complexes et ambigus (NLP). 

L'approche **"compromis partout"** reflète la réalité professionnelle où chaque choix technique implique des trade-offs entre vitesse, précision, mémoire et complexité.

La philosophie **"pas de solution miracle"** prépare les étudiants à l'adaptabilité nécessaire en environnement professionnel, où les contraintes métier dictent les choix techniques plutôt que les préférences algorithmiques.

L'intégration réfléchie des LLM dans le workflow pédagogique montre comment évoluer avec les outils modernes tout en conservant une compréhension fondamentale des principes sous-jacents.

[1](https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/attachments/76884410/49454eeb-fcd1-4b2d-aaa9-b6c7bb3474a8/transcriptM07D04.txt)
[2](https://www.ibm.com/think/topics/natural-language-processing)
[3](https://blog.octo.com/nettoyage-du-texte-en-nlp-moins-de-vocabulaire-moins-de-bruit)
[4](https://seonorth.ca/fr/nlp/stemming-and-lemmatization/)
[5](https://codefinity.com/fr/courses/v2/c68c1f2e-2c90-4d5d-8db9-1e97ca89d15e/026d736a-1860-4e3e-a915-b926ca2d9ed8/9f624800-92f4-4e6d-bb3d-40e002209bd5)
[6](https://www.learndatasci.com/glossary/tf-idf-term-frequency-inverse-document-frequency/)
[7](https://www.lyzr.ai/glossaries/tokenization/)
[8](https://www.deeplearning.ai/resources/natural-language-processing/)
[9](https://www.geeksforgeeks.org/machine-learning/understanding-tf-idf-term-frequency-inverse-document-frequency/)
[10](https://neptune.ai/blog/tokenization-in-nlp)
[11](https://www.qualtrics.com/experience-management/customer/natural-language-processing/)
[12](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
[13](https://codefinity.com/blog/A-Comprehensive-Guide-to-Text-Preprocessing-with-NLTK)
[14](https://en.wikipedia.org/wiki/Natural_language_processing)
[15](https://fr.wikipedia.org/wiki/TF-IDF)
[16](https://www.geeksforgeeks.org/nlp/nlp-how-tokenizing-text-sentence-words-works/)
[17](https://datascientest.com/en/natural-language-processing-definition-and-principles)
[18](https://datascientest.com/tf-idf-intelligence-artificielle)
[19](https://www.codecademy.com/learn/dsnlp-text-preprocessing/modules/nlp-text-preprocessing/cheatsheet)
[20](https://www.openstudio.fr/metiers/intelligence-artificielle/natural-language-processing-nlp/)
[21](https://cuik.io/blog/lexique-seo/tf-idf-term-frequency-inverse-document-frequency/)
[22](https://www.kaggle.com/code/satishgunjal/tokenization-in-nlp)
[23](https://aws.amazon.com/fr/what-is/nlp/)
[24](https://programminghistorian.org/fr/lecons/analyse-de-documents-avec-tfidf)
[25](https://milvus.io/ai-quick-reference/what-are-stop-words-in-nlp)
[26](https://www.cambridgespark.com/blog/word-embeddings-in-python)
[27](https://codesignal.com/learn/courses/text-data-preprocessing-in-python/lessons/demystifying-stop-words-in-natural-language-processing)
[28](https://www.shanelynn.ie/word-embeddings-in-python-with-spacy-and-gensim/)
[29](https://fr.mathworks.com/discovery/stemming.html)
[30](https://stackoverflow.com/questions/73568510/spacy-models-with-different-word2vec-embeddings-give-same-results)
[31](https://nlp.stanford.edu/IR-book/html/htmledition/dropping-common-terms-stop-words-1.html)
[32](https://fr.eitca.org/artificial-intelligence/eitc-ai-dltf-deep-learning-with-tensorflow/tensorflow/processing-data/examination-review-processing-data/what-is-the-difference-between-lemmatization-and-stemming-in-text-processing/)
[33](https://spacy.io/usage/embeddings-transformers)
[34](https://gist.github.com/sebleier/554280)
[35](https://www.ibm.com/fr-fr/think/topics/stemming-lemmatization)
[36](https://ashutoshtripathi.com/2020/09/04/word2vec-and-semantic-similarity-using-spacy-nlp-spacy-series-part-7/)
[37](https://github.com/stopwords-iso/stopwords-fr)
[38](https://fr.linkedin.com/advice/1/how-do-you-incorporate-stemming-lemmatization?lang=fr&lang=fr)
[39](https://spacy.io/usage/spacy-101)
[40](https://www.kaggle.com/datasets/rtatman/stopword-lists-for-19-languages)
[41](https://www.geeksforgeeks.org/nlp/lemmatization-vs-stemming-a-deep-dive-into-nlps-text-normalization-techniques/)